In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate -U

In [2]:
from google.colab import drive
import pandas as pd
import os
import json
import re
from datasets import Dataset
from torch.utils.data import DataLoader

from transformers import T5ForConditionalGeneration, T5Tokenizer, MT5ForConditionalGeneration, MT5Tokenizer
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer
os.environ['WANDB_SILENT']="true"
os.environ["WANDB_DISABLED"] = "true"

In [3]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/Shared-Tasks/Num-Eval

/content/drive/MyDrive/Shared-Tasks/Num-Eval


In [5]:
def collator(batch):

    input = batch['t5-input'] #load original sentences
    label = batch['ans'] #load noisy sentences
    inputs = tokenizer(input, text_target=label, return_tensors="pt", max_length = 512, padding='max_length',truncation=True) #tokenized sentences

    return inputs

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_id="google/flan-t5-small"
saved_model = "./Outputs/Current_Best"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(saved_model)
# model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
from datasets import load_from_disk
test_data = load_from_disk('./Dataset/test')

In [8]:
test_tokenized = test_data.map(collator, remove_columns=test_data.column_names, batch_size=8, num_proc=4, batched=True)

In [ ]:
from tqdm import tqdm
dataloader = DataLoader(test_data, batch_size=8)

#perform inference
predictions = []
for data in tqdm(dataloader):

    inputs = tokenizer(data['t5-input'], return_tensors="pt",max_length = 512, padding='max_length',truncation=True)
    output_ids = model.generate(input_ids=inputs['input_ids'], max_length = 512)
    predictions.extend(tokenizer.batch_decode(output_ids,skip_special_tokens=True))


 11%|█         | 18/161 [07:31<58:24, 24.51s/it]

In [ ]:
count = [1 if p==t else 0 for p,t in zip(predictions,test_data['ans'])]

In [ ]:
sum(count)/len(count)

In [ ]:
test_data = test_data.add_column("predictions",predictions)

In [ ]:
df = pd.DataFrame(test_data)

In [ ]:
df.head()

In [ ]:
analysis = df[['calculation','ans','predictions','t5-input']]
analysis['operation'] =  analysis['calculation'].apply(lambda x: x.split("(")[0].strip())
missed = analysis[(analysis['ans']!=analysis['predictions'])]

In [ ]:
missed['operation'] =  missed['calculation'].apply(lambda x: x.split("(")[0].strip())

In [ ]:
missed['operation'].value_counts()/analysis['operation'].value_counts()

In [ ]:
analysis['operation'].value_counts()

In [ ]:
df.to_csv('./Outputs/Trial/predictions-base.csv',index=False)

In [ ]:
import datasets
mathQA = datasets.load_dataset("gsm8k","main")

In [ ]:
mathQA

In [ ]:
df = pd.DataFrame(mathQA['train'])

In [ ]:
df

In [ ]:
from datasets import load_from_disk
train_data = load_from_disk('./Dataset/train')

In [ ]:
train_df = pd.DataFrame(train_data)

In [ ]:
train_df[train_df['calculation'].str.contains('Copy')]

In [ ]:
preds = pd.read_csv('./Outputs/Trial/predictions-base.csv')

In [ ]:
preds[preds['calculation'].str.contains('Copy')]